## Loading packages, constants, and functions

In [1]:
import functions as f
import constants as c
import classes as cl

import pandas as pd
import os, joblib, time

from netwulf import visualize
print(f"Current directories in {c.cwd}\n {os.listdir()}")

long = c.cwd + "\\raw_data\\long_alzheimers.tsv"
short = c.cwd + "\\raw_data\\short_alzheimers.tsv"

c:\Users\tobia\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\tobia\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


Current directories in c:\Users\tobia\Vs code\02807\proj\02807_project_Group13\
 ['.git', '.gitignore', 'classes.py', 'constants.py', 'enrichment.ipynb', 'functions.py', 'joblib_vars', 'Notebook.ipynb', 'OldCode_saveforReport.ipynb', 'raw_data', 'README.md', '__pycache__']


## Loading Datasets

#### Creating id_prot and saving
id_prot is a dictionary meant to point a id <integer> to a protein name <string> with 19 str characters \
id_prot is saved using joblib.dump() in dir "joblib_vars", and can be reloaded as a dict object using joblib.load(path)

In [ ]:
chunksize = 10
info_df = pd.read_csv("https://stringdb-downloads.org/download/protein.info.v12.0/9606.protein.info.v12.0.txt.gz", compression='gzip', sep="\t")
info_df = info_df[["#string_protein_id"]].values
id_prot = dict()
for c, e in enumerate(info_df):
    id_prot[c] = e[0]
print(id_prot)
joblib.dump(id_prot, "./joblib_vars/id_prot.joblib")

#### Modifying raw datasets
modified datasets are stored in "/mod_data/" for easy retrieval

In [ ]:
"""
An example of an interaction dataset loaded
"""
alz_int_df = pd.read_csv("./raw_data/long_alzheimers.tsv", sep="\t")
alz_int_df.columns = ["Prot1_ShortName", "Prot2_ShortName", "protein1", "protein2", "col1", "col2", "col3", "col4", "col5", "col6", "col7", "col8", "col9"]
alz_int_df

In [ ]:
HS_int_df = pd.read_csv("https://stringdb-downloads.org/download/protein.links.detailed.v12.0/9606.protein.links.detailed.v12.0.txt.gz", compression="gzip", sep=" ")
HS_int_df = HS_int_df[HS_int_df["experimental"] > 0] #removing interactions with no experimental relevance 
#joblib.dump(HS_int_df, "./joblib_vars/HS_int_df") #FILE TOO LARGE 
HS_int_df


Creating HS_int_simple, containing protein1 and protein2 names converted to int using id_prot. \
If all interactions are weight=1, only the names (ids) of the proteins are needed when constructing the interaction network

In [ ]:
id_prot = joblib.load("./joblib_vars/id_prot.joblib")

#Swapping key value in dict
id_swap = dict()
for k, v in id_prot.items():
    id_swap[str(v)] = k
print(id_swap)

#exporting modified HS_int
HS_int_simple = HS_int_df.map(lambda x: id_swap[x] if x in id_swap else x).reset_index()
HS_int_simple = HS_int_simple[["protein1", "protein2"]]
joblib.dump(HS_int_simple, "./joblib_vars/HS_int_simple.joblib")


## Running the class

### Setting the color of the progress bar

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

### Running the class

In [3]:
a = cl.interaction_network()
a.create_encoding_dict(testDataset=True)
a.load_data(testDataset=True)

Creating encoding table
Fetching data
Cropping data


Parsing data:   0%|          | 0/225714 [00:00<?, ?it/s]

Data loaded in var .vertices, with size: 0.147552mb


In [ ]:
#Example code for the format of a.vertices
for root in a.vertices.keys():
    print("root", "neighbor", "norm_score")
    for neighbor in a.vertices[root].keys():
        print(root, neighbor, a.vertices[root][neighbor])
    break

In [ ]:
a.construct_graph(a.vertices)
#visualize(a.graph_network) #DANGER: large visualization - Returns html graph, aka. opens up visualization in browser.

In [ ]:
# Example usage of shortest_path
vertexes = {"A": ["B", "C"],
            "B": ["A", "C", "D"],
            "C": ["A", "B"],
            "D": ["B", "E"],
            "E": ["D"]}

# Make sure you reference the correct function name
#for start_end in (paths := f.shortest_path("A", vertexes)):
#    print(start_end, paths[start_end], sep = ":   ")

In [5]:
paths = f.shortest_path(1, a.vertices, method="dijkstra")

In [9]:
print(paths["1->1130"])
print(a.vertices[1])

['1_2086_3159_2034_1130', 0.007067137809187278]
{5446: 0.038869257950530034, 4207: 0.07302709069493522, 5206: 0.03180212014134275, 3115: 0.07302709069493522, 2086: 0.001177856301531213, 1941: 0.08244994110718493, 1058: 0.08480565371024736, 4575: 0.03180212014134275, 3063: 0.07302709069493522, 5005: 0.2826855123674912, 1130: 0.28975265017667845, 3095: 0.07302709069493522, 5750: 0.002355712603062426, 3463: 0.23792697290930506, 3631: 0.14487632508833923, 6202: 0.13780918727915195, 1131: 0.1330977620730271, 4238: 0.05653710247349823, 4644: 0.27326266195524146, 1937: 0.06595995288574794, 5050: 0.07302709069493522, 3453: 0.11189634864546526, 4979: 0.28975265017667845, 650: 0.22968197879858657, 2863: 0.13780918727915195, 5078: 0.07302709069493522, 1028: 0.10247349823321555, 3469: 0.17432273262661954, 2447: 0.08244994110718493, 3893: 0.08244994110718493, 2323: 0.05653710247349823, 2554: 0.13427561837455831, 3650: 0.07302709069493522, 3239: 0.07302709069493522, 4206: 0.07302709069493522, 2730: 

In [3]:
print(a.vertices.keys())
print(len(a.vertices))

dict_keys([1, 2, 4, 7, 10, 13, 14, 15, 19, 20, 21, 24, 25, 26, 27, 30, 33, 36, 39, 40, 42, 44, 45, 50, 60, 61, 63, 64, 65, 67, 69, 71, 75, 76, 78, 79, 80, 81, 82, 85, 86, 87, 98, 102, 103, 106, 107, 109, 114, 118, 123, 125, 126, 127, 128, 130, 131, 134, 135, 136, 138, 142, 143, 146, 147, 149, 151, 152, 153, 154, 156, 157, 159, 162, 163, 164, 165, 166, 167, 168, 171, 174, 175, 177, 178, 182, 187, 191, 192, 198, 199, 200, 204, 205, 206, 209, 212, 214, 215, 218, 221, 239, 242, 244, 248, 251, 252, 253, 256, 257, 261, 262, 264, 265, 268, 271, 273, 277, 278, 279, 280, 281, 288, 291, 293, 296, 297, 300, 301, 302, 305, 308, 316, 317, 323, 326, 327, 328, 329, 331, 332, 333, 334, 336, 337, 338, 342, 343, 344, 345, 347, 349, 351, 352, 353, 354, 356, 358, 359, 360, 361, 366, 367, 368, 370, 371, 372, 373, 374, 376, 377, 378, 379, 381, 382, 383, 384, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 399, 400, 401, 403, 405, 406, 408, 409, 411, 412, 413, 414, 416, 418, 421, 423, 424, 425, 428, 430, 4

In [ ]:
# Example usage of check_connection
vertexes = {"A": ["B", "C"],
            "B": ["A", "C", "D"],
            "C": ["A", "B"],
            "D": ["B", "E"],
            "E": ["D"],
            "F": ["G"],
            "G": ["F"]}

is_connected, connected_vertices = check_connection(vertexes)
print("The network is fully connected: {}".format(is_connected))
for i, group in enumerate(connected_vertices):
    print("Grouped vertices {}:".format(i+1))
    print(group)